In [1]:
import numpy as np
import pandas as pd
import napari
import tifffile
import skimage as ski
import scipy.ndimage as ndi
import glob
import plotly.express as px
import cellpose.models as models
import matplotlib.pyplot as plt
import cv2
import dask
import cellpose.models as models
import sutils

In [2]:
viewer = napari.Viewer()

In [3]:
model = models.Cellpose(gpu=True, model_type='cyto')

# HOMEWORK

## Part 1:  Finding a good diameter

There is a file in our files folder:  chamber1KLF5in488SOX8in647-1.tif.  Open it up and take a look in napari.  Remember you can adjust the maximum allowed contrast by right clicking on an image's contrast limits slider, and then double clicking on the numbers at the top or bottom of the range and setting them to whatever you want.

In [11]:
img = ski.io.imread('files/chamber1KLF5in488SOX8in647-1.tif')
img.shape

(3176, 3400, 3)

In [12]:
viewer.layers.clear()
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x118dbc061d0>,
 <Image layer 'Image [1]' at 0x11907455420>,
 <Image layer 'Image [2]' at 0x1190755ff70>]

Now use your cursor in napari to guess a good diameter, and try segmenting the cells with cellpose, you will have to use the "diameter", "channels" and "channel_axis" parameters.  Remember to choose one of the channels for the nucleus, and another for the cytosol.

In [18]:
cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2)
viewer.add_labels(cells, name='Diameter only')

<Labels layer 'Diameter only [2]' at 0x118dbc6dff0>

## Part 2:  Getting more cells

Even with a good choice of diameter, and a good choice for the channels, you will find cellpose is not segmenting very many of the cells.  Next try to adjust "cellprob_threshold" and "flow_threshold" to get more cells segmented.  If you get some tiny slivers of garbage, don't worry we'll take care of those later.

In [14]:
cells, flows, styles, diams = model.eval(img, diameter=300, channels=[2,3], channel_axis=2, cellprob_threshold=-2, flow_threshold=1.0)
viewer.add_labels(cells, name='Cellprob and flow')


<Labels layer 'Cellprob and flow' at 0x11915e97f10>

## Part 3:  Cleaning up the segmentation

Now we want to do two things:  clean up the garbage using our remove_objects function, choosing an area_min that throws out small stuff but keeps the cells of interest.

In [15]:
filtered_cells = sutils.remove_objects(cells, 50000, 10000000000)
viewer.add_labels(cells, name='Cellprob and flow size filtered')

<Labels layer 'Cellprob and flow size filtered' at 0x11915d84760>

Notice that our object boundaries usually stretch out beyond the actual cells.  This is because cellpose downsamples by so much when it does its segmentation, it effectively blurs.  To correct for this, and only have our segmentation cover where the cell is, we can shrink the label image using the shrink_labels function we've used before.  Play with different values of shrinkage until you think it looks good.

In [16]:
shrunk_cells = sutils.shrink_labels(filtered_cells, shrinkage=40)
viewer.add_labels(shrunk_cells, name='Shrunk cells')

<Labels layer 'Shrunk cells' at 0x118efe7ccd0>